In [0]:

df_customer = spark.sql("select * from australia_sales_and_opportunities_data.v01.customers")
df_opportunities = spark.sql("select * from australia_sales_and_opportunities_data.v01.opportunities")
df_orders = spark.sql("select * from australia_sales_and_opportunities_data.v01.orders")


In [0]:
df_customer.display()


In [0]:
df_customer.count()
df_customer_filter = df_customer.filter(df_customer.state == 'SA').select('customerid', 'customername', 'city','state')
df_customer_filter.display()




In [0]:
from pyspark.sql.functions import sum 
df_opportunities_group = df_opportunities.groupBy('customerid').agg(sum('amount').alias('total_sales'))
df_opportunities_group.display()


In [0]:
from pyspark.sql.functions import count
df_opportunities_rep = df_opportunities.groupBy('salesrep').agg(count('opportunityid').alias('opportunitiy_count')).show()


In [0]:
df_orders_500 = df_orders.filter(df_orders.orderamt > 500).select('orderid', 'customerid', 'productid', 'orderdate', 'quantity', 'orderamt', 'salesrep').show()

In [0]:
df_orders_grp =  df_orders.groupBy('salesrep').agg(sum('orderamt').alias('total_amount')).show()

In [0]:
df_orders_cnt = df_orders.groupBy('productid').agg(count('productid').alias('total_quantity')).show()

In [0]:
df_opportunities.show()

In [0]:
df_opportunities.shw = df_opportunities.filter(df_opportunities.phase == '2-Closed Won').select('*').show()

In [0]:
df_opportunities.printSchema()

In [0]:
from pyspark.sql.functions import date_format ,sum
df_opportunities_dt = df_opportunities.groupBy(date_format("date" ,"yyyy-MM").alias('month'))\
                                      .agg(sum('amount').alias('total_amount')).orderBy('month').show()

## Join customer and opportunities

In [0]:
from pyspark.sql.functions import col
cust = df_customer.alias("c")
opps = df_opportunities.alias("o")
df_customer_join = cust.join(opps, col("c.customerid") == col("o.customerid"),'inner').filter(df_customer.customerid.isNotNull())\
        .select(col("c.customerid"),col("c.customername"),col("o.opportunityid"),col("o.amount")).show()


In [0]:
from pyspark.sql.functions import col ,sum , coalesce
df_orders_join = df_orders.alias("o").join(df_opportunities.alias("op"),col("o.customerid") == col("op.customerid"),'inner').groupBy("op.customerid").agg((sum("o.orderamt") + sum("op.amount")).alias("total_sales"))\
    .select(col("op.customerid"),coalesce("total_sales")).show()